In [7]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [8]:
trips = pd.read_csv("trip.csv")
weather = pd.read_csv("weather.csv")
stations = pd.read_csv("station.csv")
# status = pd.read_csv("status.csv")

In [9]:
stations.dtypes

id                     int64
name                  object
lat                  float64
long                 float64
dock_count             int64
city                  object
installation_date     object
dtype: object

In [91]:
trips.dtypes

id                             int64
duration_sec                   int64
start_date            datetime64[ns]
start_station_name            object
start_station_id               int64
end_date              datetime64[ns]
end_station_name              object
end_station_id                 int64
bike_id                        int64
subscription_type             object
zip_code                     float64
date                  datetime64[ns]
duration_min                 float64
dtype: object

In [92]:
weather.dtypes

date                              datetime64[ns]
max_temperature_f                        float64
mean_temperature_f                       float64
min_temperature_f                        float64
max_dew_point_f                          float64
mean_dew_point_f                         float64
min_dew_point_f                          float64
max_humidity                             float64
mean_humidity                            float64
min_humidity                             float64
max_sea_level_pressure_inches            float64
mean_sea_level_pressure_inches           float64
min_sea_level_pressure_inches            float64
max_visibility_miles                     float64
mean_visibility_miles                    float64
min_visibility_miles                     float64
max_wind_Speed_mph                       float64
mean_wind_speed_mph                      float64
max_gust_speed_mph                       float64
precipitation_inches                     float64
cloud_cover         

In [12]:
# status.dtypes

In [13]:
# Cambio la columna 'installation_date', de stations, para que sea de tipo fecha.

stations.installation_date = pd.to_datetime(stations.installation_date, errors='coerce')

In [14]:
# Cambio 'start_date','end_date' a tipo fecha, y 'zip_code' a tipo numerico. Del dataframe trips

trips.start_date = pd.to_datetime(trips.start_date, errors='coerce')
trips.end_date = pd.to_datetime(trips.end_date, errors='coerce')
trips.zip_code = pd.to_numeric(trips.zip_code, errors='coerce')

In [89]:
# Cambio 'date' a tipo fecha y 'precipitation_inches' a numerico, de weather.

weather.date = pd.to_datetime(weather.date, errors='coerce')
weather.precipitation_inches = pd.to_numeric(weather.precipitation_inches, errors='coerce')

In [70]:
# Cambio 'time' de tipo fecha, de status

# status.time = pd.to_datetime(status.time, errors='coerce')

In [76]:
# Agrego una columna a trips para guardar fechas sin horas, para poder hacer operaciones entre dataframes.

trips['date'] = trips.start_date.dt.date
trips.date = pd.to_datetime(trips.date, errors='coerce')

In [18]:
# Las personas sin subscripciones hacen viajes mas largos.

trip_duration_subs = trips.loc[trips.subscription_type == 'Subscriber'].duration.mean()
trip_duration_nonsub = trips.loc[trips.subscription_type == 'Customer'].duration.mean()

print('Duracion de viajes promedio de personas con subscripcion: ' + str(trip_duration_subs))
print('Duracion de viajes promedio de personas sin subscripcion: ' + str(trip_duration_nonsub))

Duracion de viajes promedio de personas con subscripcion: 590.048856101
Duracion de viajes promedio de personas sin subscripcion: 3951.76132851


In [22]:
# Renombro duration a duration_sec, y agrego duration_min con la conversion a minutos.

trips.rename(columns={'duration' : 'duration_sec'}, inplace=True)
trips['duration_min'] = trips.duration_sec/60

,id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,date,duration_min
0,4576,63,2013-08-29 14:13:00,South Van Ness at Market,66,2013-08-29 14:14:00,South Van Ness at Market,66,520,Subscriber,94127.0,2013-08-29,1.050000
1,4607,70,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber,95138.0,2013-08-29,1.166667
2,4130,71,2013-08-29 10:16:00,Mountain View City Hall,27,2013-08-29 10:17:00,Mountain View City Hall,27,48,Subscriber,97214.0,2013-08-29,1.183333
3,4251,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber,95060.0,2013-08-29,1.283333
4,4299,83,2013-08-29 12:02:00,South Van Ness at Market,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber,94103.0,2013-08-29,1.383333


In [36]:
# Cuánta gente se 'robó' bicicletas
# (Consideramos robada cuando tardó + de 1 semana en devolver)

diasDistintos = trips.loc[trips.start_date.dt.date != trips.end_date.dt.date, :]
unaSemana = 60 * 24 * 7
diasDistintos.loc[diasDistintos.duration_min > unaSemana, :].head(3)

,id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,date,duration_min
80510,111309,722236,2013-11-30 13:29:00,University and Emerson,35,2013-12-08 22:06:00,University and Emerson,35,247,Customer,94301.0,2013-11-30,12037.266667
93400,129504,619322,2013-12-18 09:16:00,San Jose Diridon Caltrain Station,2,2013-12-25 13:18:00,SJSU 4th at San Carlos,12,653,Subscriber,94041.0,2013-12-18,10322.033333
195379,361321,715339,2014-07-13 05:50:00,Arena Green / SAP Center,14,2014-07-21 12:32:00,Adobe on Almaden,5,251,Customer,NaN,2014-07-13,11922.316667


In [96]:
trips.join(weather, lsuffix='_t', rsuffix='_w', on='date').head(5)

,id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code_w
0,4576,63,2013-08-29 14:13:00,South Van Ness at Market,66,2013-08-29 14:14:00,South Van Ness at Market,66,520,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4607,70,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4130,71,2013-08-29 10:16:00,Mountain View City Hall,27,2013-08-29 10:17:00,Mountain View City Hall,27,48,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4251,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4299,83,2013-08-29 12:02:00,South Van Ness at Market,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
